In [3]:
from model_tracking import ModelTrack
db_uri = "sqlite:///Users/leepand/Downloads/MLOps/AlgoLink/tests/sql.db"
from sqlalchemy import create_engine
_engine = create_engine(db_uri)
test = ModelTrack("nick_name","project_name",_engine)

def test_1(x):
    model_log = ModelTrack('test项目', 'test备注',_engine)
    # 训练参数
    tf_param = {}
    tf_param['model'] = 'BILSTM_Attention'
    tf_param['optimizer'] = 'adam'
    tf_param['num_classes'] = 10
    tf_param['lr'] = 0.001
    tf_param['training_steps'] = 1000000
    tf_param['display_step'] = 1
    tf_param['batch_size'] = 100
    tf_param['num_hidden'] = 128
    tf_param['embedding_dim'] = 128
    tf_param['drop_out'] = 0.5
    model_log.add_model_name('bilstm_crf')
    model_log.add_model_remark('bilstm_crf备注')
    model_log.add_param(tf_param, 'tf_param')
    for i in range(1, 21):
        model_log.add_metric('train_loss', 25.0 - i, i)
        model_log.add_metric('test_loss', 24.0 - i, i)
        model_log.add_metric('test_acc', 0.3 + 0.02 * i, i)
    model_log.add_best_result('best_loss', 2.34, 20)
    model_log.add_best_result('best_acc', 0.99, 20)
    
test_1("x")

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/e3q8)

In [3]:
from functools import wraps
from algolink.core.objects.core import AlgoLinkObject,_with_meta,Project
from algolink.utils.index_dict import IndexDict, IndexDictAccessor
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
from pyjackson.decorators import make_string, type_field
import datetime

def _with_meta2(saved=True):
    """
    Decorator for methods to check that object is binded to meta repo

    :param saved: method to apply decorator
    """

    def dec(method):
        @wraps(method)
        def inner(self: AlgoLinkObject, *args, **kwargs):
            self._check_meta(saved)
            return method(self, *args, **kwargs)

        return inner

    return dec(saved) if callable(saved) else dec

@make_string('id', 'name')
class te(AlgoLinkObject):
    def __init__(self,name: str, id: int = None, author: str = None, creation_date: datetime.datetime = None):
        print (self._meta)
        super().__init__(id, name,author,creation_date)
        self._tasks: IndexDict[Task] = IndexDict('id', 'name')
        print (self._meta)
    #@_with_meta
    def hh(self,l):
        print (self._meta)
        return l
    
g = te("dd",1)
g.hh("ff")

None
None
None


'ff'

In [28]:
Project("dd",1)

Project(id=1,name=dd)

In [4]:
from algolink.core.objects.core import AlgoLinkObject
from pyjackson.decorators import make_string, type_field


In [5]:
def check_str(s, type):
    if not isinstance(s, str):
        raise Exception(type + ' not string!')
    elif s == '':
        raise Exception(type + ' is null!')

class SubModel(AlgoLinkObject):
    def __init__(self,name: str=None,id: int=None,
                 author:str=None,
                 experiment_id: int=None,
                 creation_date: datetime.datetime = None):
        super().__init__(id, name, author,creation_date)
        self.experiment_id = experiment_id
        self.name = name
        
class ModelMetric(AlgoLinkObject):
    def __init__(self,name: str=None,
                 author:str=None,
                 metric_value: float=None,
                 epoch: int=None,
                 sub_model_id: int=None,
                 creation_date: datetime.datetime = None):
        super().__init__(id, name, author,creation_date)
        self.metric_name= name
        self.metric_value = metric_value
        self.epoch = epoch
    
class BestResult(AlgoLinkObject):
    def __init__(self,name: str=None,
                 author:str=None,
                 metric_value: float=None,
                 epoch: int=None,
                 creation_date: datetime.datetime = None):
        super().__init__(id, name, author,creation_date)
        self.best_name= name
        self.best_value = metric_value
        self.best_epoch = epoch

@make_string('id', 'name')
class Experiment(AlgoLinkObject):
    """
    Experiment is a collection of submodels

    :param id: experiment id
    :param name: experiment name
    :param author: user that created that experiment
    :param creation_date: date when this experiment was created
    """
    _nested_fields_meta = _nested_fields_art = ['_submodels']

    def __init__(self, name: str, id: int = None, 
                 author: str = None, 
                 project_remark: str = None,
                 nick_name: str = None,
                 creation_date: datetime.datetime = None):
        super().__init__(id, name, author,creation_date)
        self.project_remark = project_remark
        self.model_name = ''
        self.param_dict = {}
        self.remark = ''
        self._submodels: IndexDict[SubModel] = IndexDict('id', 'name')
        self.submodels: IndexDictAccessor[SubModel] = IndexDictAccessor(self._submodels)
    """
    :param model_name: str，模型名称
    """
    def add_model_name(self, model_name):

        check_str(model_name, 'model_name')
        self.sub_model_id = 1
        self.model_name = model_name
    """
    :param remark: str，模型备注
    """
    def add_model_remark(self, remark):

        check_str(remark, 'remark')

        self.remark = remark
    def add_submodel(self, submodel: 'SubModel'):
        """
        Add submodel to experiment and save it to meta repo

        :param submodel: submodel to add
        """
        if submodel.experiment_id is not None and submodel.experiment_id != self.id:
            raise errors.MetadataError('Submodel is already in Experiment {}. Delete it first'.format(submodel.experiment_id))

        submodel.experiment_id = self.id
        self.sub_model_id = submodel.id
        self._submodels.add(submodel)
        print(self._submodels)
        return submodel.bind_as(self)
    """
    :param param_dict: dict，训练参数字典
    :param param_type: str，参数类型，例如：TF参数、word2vec参数等。
    """
    def add_param(self, param_dict, param_type):

        check_str(param_type, 'param_type')

        self.param_dict[param_type] = param_dict

    """
    :param metric_name:  str，评估指标名称，可选择['train_loss', 'test_loss', 'test_acc', 'test_recall', 'test_precision', 'test_F1']
    :param metric_value: float，评估指标数值。
    :param epoch:        int，训练周期
    第一次调用该 API 时，会把以上设置的数据持久化到 SQLite 数据库。
    可以在每个 epoch 周期的最后使用该 API添加训练集和测试集的评估指标，web端会自动获取该数据。
    """
    def add_metric(self, metric_name,metric_value,epoch):
        model_metric = ModelMetric(name = metric_name,metric_value=metric_value,epoch=epoch)
        #check_str(metric_name, 'metric_name')

        #if metric_name not in ['train_loss', 'test_loss', 'test_acc', 'test_recall', 'test_precision', 'test_F1']:
        #    raise Exception("Your metric_name：%s, not in ['train_loss', 'test_loss', 'test_acc', 'test_recall', 'test_precision', 'test_F1']" % (metric_name))

        model_metric.sub_model_id = self.sub_model_id
    
    """
    :param best_name:  str，最佳评估指标名称，
    :param best_value: float，最佳评估指标数值。
    :param best_epoch: int，训练周期
    添加当前模型训练中最佳的评估数据，一般放到模型训练的最后进行添加。
    """
    def add_best_result(self, best_name, best_value, best_epoch):

        check_str(best_name, 'best_name')
print(type(Experiment("zd",'f').add_submodel(SubModel())),'f')


<IndexDict>{None[None]=<__main__.SubModel object at 0x7f93e03ab850>}
<class '__main__.SubModel'> f


In [84]:
e=Experiment("zd",'f',project_remark="jh")
tf_param = {"d":1}
e.add_param(tf_param, 'tf_param')
e.param_dict

{'tf_param': {'d': 1}}

In [87]:
def test_1(x):
    #model_log = ModelTrack('test项目', 'test备注',_engine)
    model_log = Experiment("zd",'f',project_remark="jh")
    # 训练参数
    tf_param = {}
    tf_param['model'] = 'BILSTM_Attention'
    tf_param['optimizer'] = 'adam'
    tf_param['num_classes'] = 10
    tf_param['lr'] = 0.001
    tf_param['training_steps'] = 1000000
    tf_param['display_step'] = 1
    tf_param['batch_size'] = 100
    tf_param['num_hidden'] = 128
    tf_param['embedding_dim'] = 128
    tf_param['drop_out'] = 0.5
    model_log.add_model_name('bilstm_crf')
    model_log.add_model_remark('bilstm_crf备注')
    model_log.add_param(tf_param, 'tf_param')
    for i in range(1, 21):
        model_log.add_metric('train_loss', 25.0 - i, i)
        model_log.add_metric('test_loss', 24.0 - i, i)
        model_log.add_metric('test_acc', 0.3 + 0.02 * i, i)
    model_log.add_best_result('best_loss', 2.34, 20)
    model_log.add_best_result('best_acc', 0.99, 20)
    return model_log
    
test_1("x").param_dict

{'tf_param': {'model': 'BILSTM_Attention',
  'optimizer': 'adam',
  'num_classes': 10,
  'lr': 0.001,
  'training_steps': 1000000,
  'display_step': 1,
  'batch_size': 100,
  'num_hidden': 128,
  'embedding_dim': 128,
  'drop_out': 0.5}}

In [18]:
import os
import shutil
from typing import Dict, List, Optional, TypeVar, Union

from pyjackson import read, write
from pyjackson.utils import resolve_subtype

from algolink.core.errors import ExistingImageError, ExistingInstanceError
from algolink.core.objects import Image, Model, Pipeline, RuntimeEnvironment, RuntimeInstance, Task
from algolink.core.objects.core import AlgoLinkObject, Project, WithDatasetRepository
from algolink.core.objects.dataset_source import Dataset
from algolink.repository import DatasetRepository
from algolink.repository.artifact import ArtifactRepository
from algolink.repository.artifact.inmemory import InMemoryArtifactRepository
from algolink.repository.artifact.local import LocalArtifactRepository
from algolink.repository.dataset.artifact import ArtifactDatasetRepository
from algolink.repository.metadata import MetadataRepository
from algolink.repository.metadata.base import ProjectVar, TaskVar
from algolink.repository.metadata.local import LocalMetadataRepository
from algolink.runtime.server import Server
from algolink.utils.importing import module_importable

T = TypeVar('T', bound=AlgoLinkObject)

class AlgoLink2:
    """Main entry point for algolink

    This is the client for AlgoLink API. It can save, load and build Models, Tasks and Projects.
    AlgoLink instance can be obtained from factory methods like :meth:`~algolink.AlgoLink.local` for local client,
    :meth:`~algolink.AlgoLink.inmemory` for inmemory client.

    You can save client config with :meth:`~algolink.AlgoLink.save_client_config` and
    later restore it with :meth:`~algolink.AlgoLink.from_config_file`

    :param meta_repo: :class:`~algolink.repository.MetadataRepository` instance to save metadata
    :param artifact_repo: :class:`~algolink.repository.ArtifactRepository` instance to save artifacts
    """

    #default_server: Server = None
    #default_env: RuntimeEnvironment = None

    def __init__(self, meta_repo: MetadataRepository, artifact_repo: ArtifactRepository,
                 dataset_repo: DatasetRepository = None):
        self.meta_repo = meta_repo
        self.artifact_repo = artifact_repo
        self.dataset_repo = dataset_repo or ArtifactDatasetRepository(self.artifact_repo)

    def _bind(self, obj: Optional[Union[T, List[T]]]) -> Optional[Union[T, List[T]]]:
        """Binds AlgoLinkObject to meta and art repo of this client instance

        :param obj: subclass of AlgoLinkObject instance or list of them
        """
        if obj is None:
            return
        if isinstance(obj, list):
            for o in obj:
                self._bind(o)
        else:

            obj.bind_meta_repo(self.meta_repo).bind_artifact_repo(self.artifact_repo)
            if isinstance(obj, WithDatasetRepository):
                obj.bind_dataset_repo(self.dataset_repo)
        return obj
    
    def get_or_create_experiment(self, experiment: str, submodel_name: str) -> SubModel:
        """
        Creates a experiment if not exists or gets existing submodel otherwise.

        :param project: experiment to search/create submodel in
        :param submodel_name: expected name of submodel
        :return: created/found submodel
        """
        return self._bind(self.meta_repo.get_or_create_submodel(experiment, submodel_name))
    
    
AlgoLink2.get_or_create_experiment(experiment="E",submodel_name = "submodel")

TypeError: get_or_create_experiment() missing 1 required positional argument: 'self'

In [20]:
class tt(object):
    @classmethod
    def get_kwargs(cls, task: Task) -> dict:
        return dict(id=task.id,
                    name=task.name,
                    author=task.author)
    

class Task(object):
    def __init__(self,id=1,name="d",author="a"):
        self.id=id
        self.name=name
        self.author=author
        

kwargs = tt.get_kwargs(Task())

In [26]:
def update_attrs(obj, **attrs):
    for name, value in attrs.items():
        setattr(obj, name, value)
        
m=Task
update_attrs(m,**kwargs)

In [28]:
kwargs

{'id': 1, 'name': 'd', 'author': 'a'}

In [31]:
id(2)

4359881888

In [35]:
m.author

'a'

In [26]:
import contextlib
from algolink.utils.log import logger
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
from algolink.ext.sqlalchemy.models import Base,TExperiment,Attaching,SProject
import time
from typing import List, Optional, Type, TypeVar, Union
from pyjackson.decorators import make_string, type_field
from algolink.repository.metadata.base import ProjectVar, TaskVar, bind_to_self

@make_string('id', 'name')
class Experiment:
    def __init__(self, db_uri: str):
        self.db_uri = db_uri
        self._engine = create_engine(db_uri)
        Base.metadata.create_all(self._engine)
        self._Session = sessionmaker(bind=self._engine)
        self._active_session = None
    @contextlib.contextmanager
    def _session(self) -> Session:
        if self._active_session is None:
            logger.debug('Creating session for %s', self.db_uri)
            self._active_session = self._Session()
            new_session = True
        else:
            new_session = False

        try:
            yield self._active_session

            if new_session:
                self._active_session.commit()
        except:  # noqa
            if new_session:
                self._active_session.rollback()
            raise
        finally:
            if new_session:
                self._active_session.close()
                self._active_session = None
    def _create_exp(self,obj):
        with self._session() as s:
            p = obj
            s.add(p)
            try:
                logger.debug('Inserting object %s', p)
                s.commit()
            except IntegrityError:
                raise error_type(obj)
            return obj
    def _get_sql_object_by_id(self, object_type: Type[Attaching], id: int):
        with self._session() as s:
            #logger.debug('Getting %s[%s]', object_type.__name__, id)
            #print(object_type.id)
            obj = s.query(object_type).filter(object_type.id == id).first()
            print(obj.name,"obj")
            if obj is None:
                return
            return obj.to_obj()
    @bind_to_self
    def get_project_by_id(self,obj, id: int):# -> Optional[Project]:
        return self._get_sql_object_by_id(obj, id)
    def __repr__(self):
        return """Project '{name}', tasks""".format(name=self.name)

exp=Experiment(db_uri="sqlite:///sql.db")
tt=TExperiment(id=1)

ui=exp._get_sql_object_by_id(SProject,1)

my_project obj


In [34]:
from pyjackson import serialize
from pprint import pprint
pprint(serialize(ui))

{'author': 'leepand',
 'creation_date': '2021-10-05 14:19:53.778422 ',
 'id': 1,
 'name': 'my_project'}


In [41]:
for u in ui._tasks.items():
    print(u)

(1, Task(id=1,name=my_task2))
(2, Task(id=2,name=my_task3))
(3, Task(id=3,name=my_task))
(4, Task(id=4,name=my_task_churn_model))


In [3]:
import datetime
@make_string('id', 'name')
class TExperiment(object):
    def __init__(self, name: str, id: int = None, author: str = None,
                 sub_model_remark:str=None,
                 nick_name:str=None,
                 sub_model_sequence:int=None,
                 task_id:int=None,
                 creation_date: datetime.datetime = None):
        self.id=id
        self.name=name
        self.author=author
        self.sub_model_remark=sub_model_remark
        self.nick_name=nick_name
        self.task_id=task_id
        self.creation_date=creation_date
        #super().__init__(id, name, author, creation_date)   

In [46]:
exp._create_exp(TExperiment(name="first_experiment",
                            author="leepand",
                            sub_model_remark="churn model experiment tracking",
                            nick_name="churn_model_exp",
                            sub_model_sequence=1,
                            creation_date = datetime.datetime.utcnow(),
                            task_id=1))

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [15]:
from  ebonite.ext.sqlalchemy.repository import SQLAlchemyMetaRepository
from   ebonite.ext.sqlalchemy.models import SProject
from algolink.core.errors import ExistingProjectError
te=SQLAlchemyMetaRepository(db_uri="sqlite:///sql.db")._create_object(SProject(2,"new","dd",1),Project,ExistingProjectError)

TypeError: __init__() takes 1 positional argument but 5 were given

In [33]:
import getpass
getpass.getuser()

'leepand'

In [31]:
import datetime
datetime.datetime.utcnow()

datetime.datetime(2021, 10, 7, 12, 57, 14, 128091)

In [18]:
SProject.to_obj()

TypeError: to_obj() missing 1 required positional argument: 'self'

In [37]:
from ebonite.repository.metadata import MetadataRepository
MetadataRepository.get_or_create_project(name="my_project")

TypeError: get_or_create_project() missing 1 required positional argument: 'self'

In [2]:
from pyjackson.utils import resolve_subtype
from algolink.repository.metadata import MetadataRepository
metadata_type = resolve_subtype(MetadataRepository, {'type': "sqlalchemy"})
meta_kwargs={'db_uri': 'sqlite:///sql.db'}
meta_kwargs = meta_kwargs or {}
metadata = metadata_type(**meta_kwargs)

In [9]:
import datetime
class Project(object):
    """
    Project is a collection of tasks
    :param id: project id
    :param name: project name
    :param author: user that created that project
    :param creation_date: date when this project was created
    """
    _nested_fields_meta = _nested_fields_art = ['_tasks']

    def __init__(self, name: str, id: int = None, author: str = None, creation_date: datetime.datetime = None):
        self.name=name
        self.id=id
        self.author=author
        self.creation_date=creation_date
        #super().__init__(id, name, author, creation_date)
metadata.create_project(Project(2,"new","dd",1))

AttributeError: 'Project' object has no attribute 'tasks'

In [ ]:
project_id, sub_model_count + 1, model_name, self.remark, self.nick_name,